In [7]:
import tempfile
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import re ,shutil
from urllib.request import urlopen
import os

def get_file_path(filename):
    return os.path.join(tempfile.gettempdir(), filename)
def load(Path):   
    
    appended_data = []
    url = "http://nemweb.com.au/Reports/Current/Dispatch_SCADA/"
    result = urlopen(url).read().decode('utf-8')
    pattern = re.compile(r'[\w.]*.zip')
    filelist1 = pattern.findall(result)
    filelist_unique = dict.fromkeys(filelist1)
    filelist_sorted=sorted(filelist_unique, reverse=True)
    filelist = filelist_sorted[:600]
    try:
        df = ds.dataset(Path + "/aemo/log/scada/part-0.parquet").to_table().to_pandas()
    except:
        df=pd.DataFrame(columns=['file'])     
    file_loaded= df['file'].unique()
    #print (df)

    current = file_loaded.tolist()
    #print(current)

    files_to_upload = list(set(filelist) - set(current))
    files_to_upload = list(dict.fromkeys(files_to_upload)) 
    print(str(len(files_to_upload)) + ' New File Loaded')
    if len(files_to_upload) != 0 :
      for x in files_to_upload:
            with urlopen(url+x) as source, open(get_file_path(x), 'w+b') as target:
                shutil.copyfileobj(source, target)
            df = pd.read_csv(get_file_path(x),skiprows=1,usecols=["SETTLEMENTDATE", "DUID", "SCADAVALUE"],parse_dates=["SETTLEMENTDATE"])
            df=df.dropna(how='all') #drop na
            df['SETTLEMENTDATE']= pd.to_datetime(df['SETTLEMENTDATE'])
            df['Date'] = df['SETTLEMENTDATE'].dt.date
            df['file'] = x
            appended_data.append(df)
            # see pd.concat documentation for more info
      appended_data = pd.concat(appended_data)
      existing_file = pd.DataFrame( file_loaded)
      new_file = pd.DataFrame(  appended_data['file'].unique())
      log = pd.concat ([new_file,existing_file], ignore_index=True)
      #print(log)
      log.rename(columns={0: 'file'}, inplace=True)
      
      log_tb=pa.Table.from_pandas(log,preserve_index=False)
      #print(log_tb)
      log_schema = pa.schema([pa.field('file', pa.string())])
      log_tb=log_tb.cast(target_schema=log_schema)
      
      tb=pa.Table.from_pandas(appended_data,preserve_index=False)
      my_schema = pa.schema([
                      pa.field('SETTLEMENTDATE', pa.timestamp('us')),
                      pa.field('DUID', pa.string()),
                      pa.field('SCADAVALUE', pa.float64()),
                      pa.field('Date', pa.date32()),
                      pa.field('file', pa.string())
                      ]
                                                       )
      xx=tb.cast(target_schema=my_schema)
      parquet_file_name=str((appended_data['SETTLEMENTDATE'].max()).strftime('%Y%m%d%X')).replace(":","") + '{i}.parquet'
      ds.write_dataset(xx,Path + '/aemo/scada', format="parquet" , partitioning=['Date'],\
      partitioning_flavor="hive",basename_template =parquet_file_name, max_rows_per_group=120000,existing_data_behavior="overwrite_or_ignore")   
      ds.write_dataset(log_tb,Path +"/aemo/log/scada", format="parquet" ,existing_data_behavior="overwrite_or_ignore")
      return "done"
load("/lakehouse/default/Files")

StatementMeta(, 8996c09a-7859-41de-9038-a9bb61fa8433, 9, Submitted, Running)

77 New File Loaded


In [ ]:
spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
from pyspark.sql.types import *
df = spark.read.parquet('Files/aemo/scada/*/*.parquet')
df.write.mode("overwrite").format("delta").save("Tables/scada")

StatementMeta(, , , Waiting, )